<a href="https://colab.research.google.com/github/Center-for-Atmospheric-Research-ATMOS/deposition-calculator/blob/main/deposition_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Here you can estimate the coverage of your sample in the electrostatic precipitator.

In [1]:
#@markdown # Imports
# interaction
from google.colab import files

# calculations
import numpy as np
import pandas as pd
import random
from scipy.integrate import simps
from scipy.optimize import curve_fit
import scipy.stats as stats

# work with text
import re

# visualisation
import plotly.express as px

In [109]:
#@markdown # Upload file
#@markdown Run the cell to upload particle size distribution from AIM in *.TXT
#@markdown format.

datafile = files.upload();

Saving 20180704_AS_2.txt to 20180704_AS_2.txt


# Check particle size distribution

In [110]:
#@markdown Function to read file
def read_file(filename, column=1):
    '''Retrieves info from the AIM txt file.
    in the returned dataframe diameter is [nm], counts are [particles/cm^3]

    Args:
    filename: last file uploaded to google drive
    column: chosen column with counts, 1 by default

    Returns:
    df - pd.dataframe containing particle size distribution
    '''
    # read file
    filename = list(datafile.keys())[0]
    with open(filename, 'r', errors='ignore') as f:
        text = f.read()

    # get bins from AIM
    match = re.search(
        r'(?<=Diameter Midpoint\n)(.|\n)*?(?=\nScan Up Time\(s\))',
        text)
    match = match.group().strip().replace('\t\t', '\t')

    # convert bins to float, add bins to df
    df = pd.DataFrame(
        [i.split('\t') for i in match.split('\n')]).astype('float').dropna()

    print(f"Data contains {len(df.columns) - 1} columns")
    print(f"Here the column {column} is shown\n")
    df = df[[0, column]]

    # rename columns
    df = df.rename(columns={0: "Diameter", column: "Counts"})
    return df

In [111]:
#@markdown Function describing lognormal probability density
def lognormal_pdf(x, mu, sigma, scale):
    """Calculates the lognormal probability density function.

    Args:
    x: A NumPy array of particle diameters.
    mu: The mean of the lognormal distribution.
    sigma: The standard deviation of the lognormal distribution.
    scale: The factor that affecting the amplitude the distribution

    Returns:
    A NumPy array of the lognormal probability density function.
    """
    return  scale / (x * sigma * np.sqrt(2 * np.pi)) * np.exp(-(np.log10(x) - mu)**2 / (2 * sigma**2))

In [112]:
#@markdown Charge correction (if needed)

def charge_correction(Dp):
    '''Applies the formula from Wiedensohler, 1988
    coefficients are for +1 charged particles

    Args:
    Dp: array of particle diameters

    Returns:
    dN_corr: corrected dN
    '''
    a0 = -2.3484
    a1 = 0.6044
    a2 = 0.48
    a3 = 0.0013
    a4 = -0.1544
    a5 = 0.0320

    return 10**(a0 + a1 * np.log10(Dp) + a2 * np.log10(Dp)**2 + a3 * np.log10(Dp)**3\
     + a4 * np.log10(Dp)**4 + a5 * np.log10(Dp)**5)

In [114]:
# read file
column = 4
filename = list(datafile.keys())[0]
size_distribution = read_file(filename, column)

# get diameter and counts to make code clearer
Dp = np.array(size_distribution["Diameter"])
dNdlogDp = np.array(size_distribution["Counts"])

# Calculate the mean logarithmic width of each size bin
dlogDp = np.mean(np.log10(Dp[1:]) - np.log10(Dp[:-1]))

# Multiply the dN/dlog(Dp) values by the logarithmic width to get dN
dN_raw = dNdlogDp * dlogDp

# charge correction
#dN_raw = dN_raw / charge_correction(Dp)

size_distribution["dN_raw"] = dN_raw
size_distribution["dN"] = dN_raw

# plot figure
fig = px.scatter(x=Dp, y=dN_raw, log_x=True)

fig.update_layout(
    xaxis_title="Dp [nm]",
    yaxis_title="dN [particles/cm^3]",
    title="Particle size distribution",
    width=800,
    height=400
)

fig.show()

Data contains 4 columns
Here the column 4 is shown



If you don't see the expected one peak with lognormal distribution you can fit the data using the cell below.
For example, the distribution may look weird because of the concentration limit of CPC was hit.
To perform the fitting we will exclude bad points and fit the rest with one peak lognormal probability density function.


In [ ]:
# Dp-range to exclude. Data taken from the figure above.
# 2 ranges case
exclude_indices = (
    ((Dp >= 50) & (Dp <= 50) | (Dp >= 181) & (Dp <= 573))
    )
# 1 range case
#exclude_indices = (Dp >= 90) & (Dp <= 259)

# initital guess for fitting
# change if the fit does not converge
initial_guess = [0, 1, max(dN_raw)]

# fit a lognormal distribution
params, covariance = curve_fit(
    lognormal_pdf,
    Dp[~exclude_indices],
    dN_raw[~exclude_indices],
    p0=initial_guess)

# extract best fitting parameters
mu, sigma, scale = params

# get final dN in [particles/cm^3], add raw dN
size_distribution["dN_raw"] = dN_raw
size_distribution["dN"] = lognormal_pdf(Dp, mu, sigma, scale)

# plot result
fig = px.scatter(size_distribution,
                 x="Diameter",
                 y=["dN_raw", "dN"],
                 log_x=True)

fig.update_layout(
    xaxis_title="Dp [nm]",
    yaxis_title="dN [particles/cm^3]",
    title="Particle size distribution",
    width=800,
    height=400)

fig.show()

# Theory and methods

The electrical mobility of particles in general is:

$$Z_p = \frac{neC_c(d_p)}{3 \pi \eta d_p}$$

where $n$ is the number of elementary charges $e$ on the particle, $\eta$ is the viscosity of the medium, and $d_p$ is the particle diameter.
$C_c$ is the Cunningham slip correction which is a correction to the friction for particles between the continuum and free molecular regime:

$$C_c = 1 + \frac{2 \lambda}{d_p}(1.142 + 0.558e^{-\frac{0.999d_p}{2 \lambda}})$$

where $\lambda$ is the mean free path.

The particle charge $q=ne$ here is unknown.
But there is a way to restore $Z_p$ from the DMA measurements.

$$Z_p = \frac{q_c + q_m}{4 \pi \Lambda V}$$

where $q_c$ is the flow of sneath air at the DMA entry and $q_m$ the steath air flow at the DMA exit, $V$ is the voltage between inner and outer rods [Knutson and Whitby, 1975].
$\Lambda$ is an instrumental constant given by:

$$\Lambda = \frac{L}{ln(\frac{r_i}{r_o})}$$

where $L$ is the DMA length, $r_i$ and $r_o$ the radii of the inner and outer electrods.

Equating the two ways of calculating $Z_p$, we can derive particle charge:

$$q = \frac{3}{4} \frac{(q_c + q_m) \eta d_p}{\Lambda V C_c(d_p)}$$

The collection efficiency of the precipitator under ideal conditions is described by Deutsch-Anderson equation.

$$eff = 1 - exp(-w\frac{A}{Q})$$

where $A$ is the collecting area of the precipitator, $Q$ is the gas flow through the precipitator.
Migration velocity $w$ is the velocity at which the particle migrates toward the collection electrode within the electrostatic precipitator:

$$w = \frac{qE_p}{3 \pi \eta d_p}$$

The Deutsch-Anderson equation is extensevely used since 1924.
But despite it scientific corectness, it neglects several important factors, such as dust reentrainment, particle size distribution, gas flow variation, and particle sneakage.
Therefore, it should only be used for making preliminary estimates of collection efficiency.
Here, the particle size distribution will be taken into account, and the collection efficiency will be calculated for each particle size.

The particle distribution $c_p(d_p)$ in AIM file is in [particles/cm$^3$].
Concentration of particles at the deposition spot:

$$c_{spot}(d_p) = c_p(d_p) \cdot eff \cdot w \cdot t $$

where $t$ is the deposition time.

Simplified equation from Preger 2020:

$$c_{spot}(d_p) = c_p(d_p) \cdot Z_p \cdot E_p \cdot t $$

To calculate total coverage, we need to divide total area covered by particles by sample area:

$$coverage = \frac{\sum c_{spot}(d_p) \cdot \pi d_p^2}{sample\_area}$$


In [91]:
#@markdown # Retrieve metadata from the file
#@markdown Constants may be input manualy below

# read file
with open(filename, 'r', errors='ignore') as f:
        text = f.read()

# inner DMA radius [m]
ri = float(
    re.findall(
        r'DMA Inner Radius\(cm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])*1e-2

# outer DMA radius [m]
ro = float(
    re.findall(
        r'DMA Outer Radius\(cm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])*1e-2

# dinamic size viscosity [Pa*s]
try:
    eta = float(
        re.findall(
            r'Reference Gas Viscosity \(Pa\*s\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
            text)[0])
except IndexError:
    eta = float(
        re.findall(
            r'Gas Viscosity \(kg/\(m\*s\)\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
            text)[0])

# mean free path [m]
MFP = float(
    re.findall(
        r'Mean Free Path \(m\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])

# sneath flow at the DMA entrance [m^3/s]
qc = float(
    re.findall(
        r'Sheath Flow\(lpm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0]) * 1e-3 / 60

# sneath flow at the DMA output [lpm]
qm = qc

# DMA characteristic length [m]
L = float(
    re.findall(
        r'DMA Characteristic Length\(cm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])*1e-2

# low voltage [V]
Vmin = float(
    re.findall(
        r'Low Voltage\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])

# high voltage [V]
Vmax = float(
    re.findall(
        r'High Voltage\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])

print(f'''
ri = {ri} [m]
ro = {ro:.2e} [m]
eta = {eta} [Pa*s]
MFP = {MFP} [m]
qc = {qc} [m^3/s]
qm = {qm} [m^3/s]
L = {L} [m]
Vmin = {Vmin} [V]
Vmax = {Vmax} [V]
''')


ri = 9.37e-05 [m]
ro = 1.96e-04 [m]
eta = 1.822e-05 [Pa*s]
MFP = 6.642e-08 [m]
qc = 6.666666666666667e-05 [m^3/s]
qm = 6.666666666666667e-05 [m^3/s]
L = 0.0044369 [m]
Vmin = 10.2125 [V]
Vmax = 9749.83 [V]



In [119]:
## set constants unavailable in the file
DEPOSITION_TIME = 1*60*60 # [s], exposure time

PLATE_DIAMETER = 1e-2 # [m], diameter of the precipitator electrode
PRECIPITATOR_D = 0.083 # [m], precipitator diameter
PRECIPITATOR_FLOW = 0.3*1e-3/60 # [m^3/s]

SAMPLE_AREA = .25*1e-4 # [m^2], sample area
Vplate = 1e4 # [V], voltage at the precipitator plate

## calculated constants
# instrumental constant [cm]
Lambda = L / np.log(ri/ro)

# electric field strength at the precipitator plate
Ep = Vplate / (PRECIPITATOR_D - PLATE_DIAMETER) * 2

# precipitator area
Ap = np.pi * PRECIPITATOR_D ** 2

In [120]:
#@markdown # Calculate coverage
# diameter is converted from [nm] to [m]
# dN is converted from [#/cm^3] to [#/m^3]

# add Cunningham slip correction
size_distribution['Cc'] = size_distribution['Diameter'].apply(
    lambda x: 1 + MFP/(x*1e-9) * (2.514 + 0.8*np.exp(-0.55*(x*1e-9)/MFP)))

# add electrical mobility, assume one charge
size_distribution['Zp'] = size_distribution.apply(
    lambda df: 1.6e-19 * 1 * df.Cc / (3 * np.pi * eta * df.Diameter * 1e-9),
    axis=1)

# add drift velocity
size_distribution['Drift_velocity'] = size_distribution.apply(
    lambda df: df.Zp * Ep,
    axis=1)

# add concentration at the spot after deposition for each particle size
size_distribution['Cspot'] = size_distribution.apply(
    lambda df: df.dN * 1e6 * df.Drift_velocity * DEPOSITION_TIME,
    axis=1)

# add total covered area for each particle size
size_distribution['Covered_area'] = size_distribution.apply(
    lambda df: df.Cspot * SAMPLE_AREA * np.pi * (df.Diameter * 1e-9)**2,
    axis=1)

# calculate total coverage
coverage = size_distribution.Covered_area.sum() / SAMPLE_AREA
print(f'Total coverage is {coverage*100:.2f}%')

# check
size_distribution

Total coverage is 10.02%


,Diameter,Counts,dN_raw,dN,Cc,Zp,Drift_velocity,Cspot,Covered_area
0,11.8,7654.480,119.489854,119.489854,19.234701,1.518812e-06,0.416113,1.789965e+11,1.957485e-09
1,12.2,4513.120,70.451820,70.451820,18.623781,1.422357e-06,0.389687,9.883491e+10,1.155367e-09
2,12.6,3419.050,53.372899,53.372899,18.051690,1.334897e-06,0.365725,7.027134e+10,8.762120e-10
3,13.1,3662.010,57.165613,57.165613,17.385764,1.236582e-06,0.338790,6.972162e+10,9.397232e-10
4,13.6,4511.430,70.425439,70.425439,16.768862,1.148855e-06,0.314755,7.980027e+10,1.159237e-09
...,...,...,...,...,...,...,...,...,...
105,514.0,723.055,11.287212,11.287212,1.326329,2.404300e-09,0.000659,2.676609e+07,5.553938e-10
106,532.8,459.584,7.174312,7.174312,1.314611,2.298970e-09,0.000630,1.626759e+07,3.626950e-10
107,552.3,397.164,6.199908,6.199908,1.303329,2.198768e-09,0.000602,1.344542e+07,3.221174e-10
108,572.5,160.570,2.506569,2.506569,1.292478,2.103528e-09,0.000576,5.200411e+06,1.338685e-10


# References


*   Instruction Manual: Model 3010 Condensation Particle Counter; 2002, TSI Incorporated, St. Paul, MN, USA.
*   Operation and Service Manual: Series 3080 Electrostatic Classifiers; 2006, TSI Incorporated, St. Paul, MN, USA.
*   E. O. Knutson and K. T. Whitby (1975). Aerosol classification by electric mobility: Apparatus, theory, and applications. J. Aerosol Sci. 6: 443-451.
*   K. Willeke and P.A. Baron: Aerosol Measurement, Van Nostrand, 1993



# Additional calculations

In [ ]:
# compare electrical mobility of 2 particles sizes

size_distribution[size_distribution.Diameter == 101.8]

,Diameter,Counts,dN_raw,dN,Cc,Zp,Drift_velocity,Cspot,Covered_area
43,101.8,110268.0,1722.407169,496.342277,2.892175,2.632046e-08,0.007211,4.295000e+09,3.495817e-09


In [ ]:
def Zp_equation(Dp, MFP, eta):
    return 1.6e-19 * 7 * (1 + MFP / (Dp * 1e-9) * (2.514 + 0.8 * np.exp(-0.55 * (Dp * 1e-9) / MFP))) / (3 * np.pi * eta * Dp * 1e-9)

def bisection_method(a, b, f, tol):
    while abs(b - a) > tol:
        midpoint = (a + b) / 2
        if f(midpoint) * f(a) < 0:
            b = midpoint
        else:
            a = midpoint
    return midpoint

target_Zp = 2.63e-8  # Target Zp value

# Set the initial interval
a = 100  # Minimum Dp value
b = 1000  # Maximum Dp value

# Find the solution using the bisection method
Dp_root = bisection_method(a, b, lambda Dp_root: Zp_equation(Dp_root, MFP, eta) - target_Zp, tol=1e-12)

print("Dp = {:.1f} nm".format(Dp_root))

Dp = 363.3 nm
